In [1]:
import joblib
from keras.models import load_model
import gradio as gr
import numpy as np

In [2]:
ann_model = load_model('ann_model.h5')
geography_encoder = joblib.load('geography_encoder.pkl')
gender_encoder = joblib.load('gender_encoder.pkl')
ct = joblib.load('column_transformer.pkl')
sc = joblib.load('standard_scaler.pkl')

In [3]:
def predict_customer_churn(CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary):
    
    IsActiveMember = int(IsActiveMember == 'True')
    HasCrCard = int(HasCrCard == 'True')

    
    new_observation = np.array([[CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, int(IsActiveMember), EstimatedSalary]])  # Convert IsActiveMember to int

    new_observation[:, 1] = geography_encoder.transform(new_observation[:, 1].reshape(-1, 1)) 
    new_observation[:, 2] = gender_encoder.transform(new_observation[:, 2].reshape(-1, 1)) 

    new_observation = ct.transform(new_observation)

    new_observation = sc.transform(new_observation)
    new_prediction = ann_model.predict(new_observation)[0]
    probability = new_prediction
    new_prediction = (new_prediction > 0.5)

    prediction_result = "Will Leave" if new_prediction else "Will Stay"
    return f"Prediction: {prediction_result}, Probability of leaving: {probability*100}%"

iface = gr.Interface(fn=predict_customer_churn, 
                     inputs=["number", "text", "text", "number", "number", "number", "number", "text", "text", "number"], 
                     outputs="text",
                     examples=[
                         [600, 'Spain', 'Female', 40, 3, 60000, 2, 'True', 'True', 50000],
                         [800, 'Germany', 'Male', 50, 5, 70000, 1, 'False', 'False', 60000]
                     ])

iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
